# Pandas DataFrame as a Database

This script creates a single dataframe containing AFU data for every structure in all experiments. 

Columns define group, drug, distance, structure, etc.

This large dataframe can be queried as needed, or exported as a CSV.

In [51]:
import os
import glob
import pandas as pd

database = pd.DataFrame(columns = ['Group', 'Distance', 'MCN', 'Sniffer', 'AFU'])

def addGroup(folderPath, groupName):    
    filePaths = glob.glob(folderPath+"/*.xls")
    filePaths = [x for x in filePaths if not "bleachCorrected" in x]
    filePaths = [x for x in filePaths if x.endswith("um.xls")]
    for filePath in filePaths:
        fileName = os.path.basename(filePath)
        distance = int(fileName.replace("um.xls", ""))
        addExperiment(filePath, groupName, distance)

def addExperiment(filePath, groupName, distance):
    global database
    df = pd.read_csv(filePath, delimiter="\t")
    roiNames = df.columns[1:]
    for roiName in roiNames:
        mcn, distance, sniffer = roiName.split(".")
        distance = int(distance.replace("um", ""))
        afu = df[roiName].values
        row = {'Group': groupName, 'Distance': distance, 'MCN': mcn, 'Sniffer':sniffer, 'AFU':afu}
        database = database.append(row,ignore_index = True)
        
addGroup(R"X:\Data\OT-Cre\OT-GCaMP-nonspecific\04-03-19 evoke OT\04-30-2020 Cs-GLU analyze", "CsGlu")
addGroup(R"X:\Data\OT-Cre\OT-GCaMP-nonspecific\04-03-19 evoke OT\04-30-2020 K-GLU analyze", "KGlu")
addGroup(R"X:\Data\OT-Cre\OT-GCaMP-nonspecific\04-03-19 evoke OT\04-30-2020 L368 - Cs analyze", "L368")

database.to_csv("all.csv")
database

,Group,Distance,MCN,Sniffer,AFU
0,CsGlu,25,19514000,C1,"[1286.65983, 1266.25717, 1189.49917, 1297.1745..."
1,CsGlu,25,19514000,C2,"[2948.64467, 2962.93833, 3239.71167, 3417.8625..."
2,CsGlu,25,19514000,C3,"[5736.4195, 5736.50717, 5026.41233, 4580.00867..."
3,CsGlu,25,19514000,C4,"[2283.93833, 2211.74517, 2253.62167, 2301.0938..."
4,CsGlu,25,19514000,C5,"[2045.98417, 1792.38467, 2021.403, 1938.1595, ..."
...,...,...,...,...,...
441,L368,125,20217028,C15,"[1024.454, 1141.794, 1079.186, 1145.165, 1190...."
442,L368,125,20217028,C16,"[1149.15, 1137.655, 1136.93, 1116.005, 1118.42..."
443,L368,125,20217028,C17,"[3113.213, 2974.022, 3031.294, 2999.846, 2674...."
444,L368,125,20217028,C18,"[1648.366, 1611.486, 1646.211, 1597.274, 1576...."
